In [2]:
!pip install flask-ngrok

In [7]:
!pip install responses

In [8]:
import numpy as np
import cv2
from flask import Flask, render_template, request, Response
from werkzeug.utils import secure_filename
import pickle
import tensorflow as tf
import numpy 
from flask_ngrok import run_with_ngrok
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import img_to_array
import os
import time
import json
import pandas as pd
import responses
import requests
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import img_to_array



In [ ]:
# model try out 

modelloaded = tf.keras.models.load_model('/content/drive/My Drive/flask api/trained_model.h5')
def convert_image_to_array(image_dir):
    default_image_size = tuple((64, 64))
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)
            img = img_to_array(image)
            img2 = np.expand_dims(img,0)
            return img2
        else :
            return np.array([])
    except Exception as e:
        print("Error :", e)
        return None



image ="/content/drive/My Drive/flask api/test2.JPG"

cag = ["Apple___Apple_scab", "Apple___Black_rot", "Apple___Cedar_apple_rust",'Apple___healthy', 'Blueberry___healthy',
           'Cherry_(including_sour)___Powdery_mildew','Cherry_(including_sour)___healthy','Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
           'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight','Corn_(maize)___healthy', 'Grape___Black_rot',
           'Grape___Esca_(Black_Measles)','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy',
           'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot','Peach___healthy', 'Pepper_bell___Bacterial_spot',
           'Potato___Early_blight', 'Potato___Late_blight','Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy',
           'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch','Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight',
           'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot',
           'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus','Tomato___healthy']
    
predict = modelloaded.predict([convert_image_to_array(image)])
x = np.where(predict == 1.)
f = int(x[1])
print(cag[f])



In [15]:
app = Flask(__name__,template_folder='/content/drive/My Drive/Plant Diagnosis/template',static_folder='/content/drive/My Drive/Plant Diagnosis/static')


run_with_ngrok(app)
@app.route('/')
def home():
    return render_template('plant1.html')

path = "/content/drive/My Drive/Plant Diagnosis/data"
orb = cv2.ORB_create(nfeatures=1000)
images = []
classname = []
myList = os.listdir(path)

cam = cv2.VideoCapture("")

for c1 in myList:
    imgCur = cv2.imread(f'{path}/{c1}',0)
    images.append(imgCur)
    classname.append(os.path.splitext(c1)[0])

def findDes(images):
    desList = []
    for img in images:
        kp,des = orb.detectAndCompute(img,None)
        desList.append(des)
    return desList
desList = findDes(images)

def findId(frame,desList):
    kp2, des2 = orb.detectAndCompute(frame, None)
    bf = cv2.BFMatcher()
    matchList= []
    finalVal = -1
    try:
        for i in desList:
            matches = bf.knnMatch(i, des2, k=2)
            good_res = []
            for j, k in matches:
                if j.distance < 0.75 * k.distance:
                    good_res.append(j)
            matchList.append(len(good_res))
    except:
        pass
    if(len(matchList)!=0):
        if(max(matchList)>=15):
            finalVal =matchList.index(max(matchList))

    return finalVal


def get_frame():
    port=0
    camera=cv2.VideoCapture(port)

    while (camera.isOpened()):
        retval, frame = camera.read()
        if retval == True:
            frame_ori = frame.copy()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            newList = []
            res_id = findId(frame, desList)
            if (res_id != -1):
                cv2.putText(frame_ori, classname[res_id], (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
                newList.append(classname[res_id])
            # send_data(newList)
            if(len(newList)!= 0):
                print(newList)
                send_data(newList)
            #print(newList)
            frame_res = cv2.imencode('.jpg', frame_ori)[1].tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame_res + b'\r\n')
            time.sleep(1)
        else:
            break


def send_data(newList):
    # open output file for writing
    with open('/content/drive/My Drive/Plant Diagnosis/listfile.txt', 'w+') as filehandle:
        json.dump(newList, filehandle)
def get_data():
    # open output file for reading
    with open('/content/drive/My Drive/Plant Diagnosis/listfile.txt', 'r') as filehandle:
        basicList = json.load(filehandle)
    return basicList


def result(var):
    df = pd.read_csv("/content/drive/My Drive/Plant Diagnosis/sampledata.csv", index_col='diease name')
    # var = 'orange diease '
    diease = df.loc[var]
    return diease


def temp():
  api_key = "1b5658d16227cbb5655c96093a3564af"
  base_url = "http://api.openweathermap.org/data/2.5/weather?"
  city_name = "dehradun"
  complete_url = base_url + "appid=" + api_key + "&q=" + city_name
  response = requests.get(complete_url)
  x = response.json()
  z = x["weather"]
  if x["cod"] != "404":
    y = x["main"]
    return y,z
  else:
    return (" City Not Found ") 

def heatindex(vTemperature,vRelativeHumidity):
  if vTemperature >= 0:
    vFeelsLike = 0.5 * (vTemperature + 61.0 + ((vTemperature-68.0)*1.2) + (vRelativeHumidity*0.094))
    if vFeelsLike >= 80:
      vFeelsLike = -42.379 + 2.04901523*vTemperature + 10.14333127*vRelativeHumidity - .22475541*vTemperature*vRelativeHumidity - .00683783*vTemperature*vTemperature - .05481717*vRelativeHumidity*vRelativeHumidity + .00122874*vTemperature*vTemperature*vRelativeHumidity + .00085282*vTemperature*vRelativeHumidity*vRelativeHumidity - .00000199*vTemperature*vTemperature*vRelativeHumidity*vRelativeHumidity
      if vRelativeHumidity < 13 and vTemperature >= 80 and vTemperature <= 112:
        vFeelsLike = vFeelsLike - ((13-vRelativeHumidity)/4)*math.sqrt((17-math.fabs(vTemperature-95.))/17)
        if vRelativeHumidity > 85 and vTemperature >= 80 and vTemperature <= 87:
          vFeelsLike = vFeelsLike + ((vRelativeHumidity-85)/10) * ((87-vTemperature)/5)				
  c= ((vFeelsLike-32) * 5/9)
  return c

@app.route('/calc')
def calc():
     return Response(get_frame(),mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/data')
def data():
    var = data1()
    print(var)
    df = pd.read_csv("/content/drive/My Drive/Plant Diagnosis/sampledata.csv", index_col='diease name')
    res = df.loc[var]
    y,z = temp()
    current_temperature = y["temp"]
    current_pressure = y["pressure"]
    current_humidiy = y["humidity"]
    weather_description = z[0]["description"]
    heat = heatindex(current_temperature,current_pressure)
    if heat > 150:
      irrigation = "the plants will have endosmosis so more water should be provided"
    elif (heat >100 and heat <150):
      irrigation = "the plants will have osmosis so more water should be provided"
    else:
      irrigation = "the plants will have endosmosis so more water should be provided"


    return render_template('plant2.html',name= var,what = res[0],how=res[1],cultural=res[2],organic=res[3],inorganic=res[4]
                           ,rate = res[5],govdata= res[6],temp=str(current_temperature),pressure =str(current_pressure),humidity =str(current_humidiy),weather =str(weather_description)
                           ,heatindex =heatindex)
      

def data1():
    str = ""
    newList = get_data()
    if (len(newList)!=0):
        res_list = list(dict.fromkeys(newList))
        for i in res_list:
            str += i
    else:
        str ="no disease found"
    return str

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e08bacd07e44.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [19/Jul/2020 16:51:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2020 16:51:14] "GET /static/boom.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2020 16:51:15] "GET /calc HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2020 16:51:16] "GET /favicon.ico HTTP/1.1" 404 -


orange


127.0.0.1 - - [19/Jul/2020 16:51:19] "GET /data HTTP/1.1" 200 -
